# Predicting prices of players from the top 5 football leagues

## Summary

## Step 1: Librairies import

In [1]:
import pandas as pd

# Beautiful Soup for Web scrapping
import requests
from bs4 import BeautifulSoup

pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns',None)

## Step 2: Data pre-processing

### Original dataset overview

In [2]:
DATA = pd.read_csv("dataset_football_cleaned.csv")
DATA

,name,first_name,age,nation,league,team,goals_selection,selections_nation,position,price,end_contract,goal_champ,assist_champ,own_goal_champ,sub_on_champ,sub_out_champ,yellow_card_champ,second_yellow_card_champ,red_card_champ,penalty_goal_champ,conceded_goal_champ,clean_sheet_champ,goal_cup,assist_cup,own_goal_cup,sub_on_cup,sub_out_cup,yellow_card_cup,second_yellow_card_cup,red_card_cup,penalty_goal_cup,conceded_goal_cup,clean_sheet_cup,goal_continent,assist_continent,own_goal_continent,sub_on_continent,sub_out_continent,yellow_card_continent,second_yellow_card_continent,red_card_continent,penalty_goal_continent,conceded_goal_continent,clean_sheet_continent
0,Etheridge,Neil,28,Philippines,PremierLeague,Cardiff,0,60,Goalkeeper,4000000.0,3.0,0.0,0.0,0.0,0.0,1.0,9.0,0.0,0.0,237.0,59.0,0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,22.0,3.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0,0
1,LéoNatel,NaN,21,Brazil,FirstDivision,APOELNicosia,0,0,RightWinger,800000.0,1.0,6.0,0.0,0.0,6.0,7.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0
2,Vidigal,André,20,Portugal,FirstDivision,APOELNicosia,0,1,RightWinger,650000.0,2.0,13.0,4.0,0.0,27.0,11.0,3.0,0.0,0.0,1.0,0.0,0,3.0,1.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,Antoniou,BaiAndrew,21,Australia,FirstDivision,AlkiOroklini,0,0,SecondStriker,50000.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,Tribeau,Yoann,30,France,FirstDivision,AlkiOroklini,0,0,AttackingMidfield,200000.0,NaN,1.0,1.0,0.0,30.0,13.0,3.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12706,Neto,Eduardo,30,Brazil,J1League,NagoyaGrampus,0,0,DefensiveMidfield,1350000.0,2.0,8.0,14.0,0.0,19.0,47.0,63.0,1.0,1.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0,0
12707,Soma,Yuki,21,Japan,J1League,NagoyaGrampus,0,0,Midfielder,75000.0,NaN,1.0,3.0,0.0,6.0,1.0,2.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
12708,Fujii,Haruya,18,Japan,J1League,NagoyaGrampus,0,0,Defender,50000.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
12709,Takeda,Yohei,31,Japan,J1League,NagoyaGrampus,0,0,Goalkeeper,125000.0,2.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,138.0,21.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,27.0,8.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


### Issue 1: Take into account only the top 5 european football leagues

In [3]:
selected_leagues = ['PremierLeague','SerieA','Bundesliga','Ligue1','LaLiga']
DATA = DATA[DATA.league.isin(selected_leagues)]

### Issue 2: Delete "fake" Premier League and BundesLiga

In [4]:
# Egyptian premier league players are after the index 4920
DATA = DATA[DATA.index<4920]
# Austrian bundesliga players are between the indexes 2702 and 2871 included
before_2072 = DATA[DATA.index<2702]
after_2871 = DATA[DATA.index>2871]
DATA = pd.concat([before_2072,after_2871],axis=0)
# Not very elegant but working

### Issue 3: Get dummies for positions

In [5]:
dum_position = pd.get_dummies(DATA['position'], columns=["position"])
DATA = DATA.drop(['position'],axis=1)
DATA = pd.concat([DATA,dum_position],axis=1)

### Issue 4: Delete goalkeepers and related variables

In [6]:
# Delete goalkeepers rows
DATA = DATA[DATA.Goalkeeper != 1]

# Delete columns related to them
DATA.drop(["Goalkeeper","conceded_goal_champ","clean_sheet_champ","conceded_goal_cup","clean_sheet_cup",
           "conceded_goal_continent","clean_sheet_continent"],axis=1,inplace=True)

### Issue 5: Sum yellow cards from championship, cup and continent (same for red cards)

In [7]:
# Create two new columns with the sum of yellow and red cards 
yellow_cards = DATA.yellow_card_champ + DATA.yellow_card_continent + DATA.yellow_card_cup
red_cards = DATA.red_card_champ + DATA.red_card_continent + DATA.red_card_cup 
cards = pd.DataFrame({"yellows_cards":yellow_cards,"red_cards":red_cards})

# Drop the old columns 
DATA.drop(["yellow_card_champ","yellow_card_continent","yellow_card_cup",
           "red_card_champ","red_card_continent","red_card_cup",
           "second_yellow_card_cup","second_yellow_card_champ","second_yellow_card_continent"],axis=1,inplace=True)

# Concat the two new columns 
DATA = pd.concat([DATA,cards],axis=1)

### Issue 6: Take out the sub on/out columns

In [8]:
DATA.drop(["sub_on_champ","sub_out_champ",
           "sub_on_cup","sub_out_cup",
           "sub_on_continent","sub_out_continent"],axis=1,inplace=True)

### Issue 7: Take out the identifiers

In [9]:
DATA.drop(["name","first_name"],axis=1,inplace=True)

## Step 3: Qualitative variables valuation using web scrapping

### 1. Team valuation using UEFA index

In [10]:
# Web scrapping on a website displaying the UEFA index on the last 5 years in English
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

page = "https://kassiesa.net/uefa/data/method5/trank2020.html"
pageTree = requests.get(page, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

# Finding the values we are looking for
teams = pageSoup.find_all("td",{'align': 'left'})
scores = pageSoup.find_all("th",{'class':'lgray'})

# Creating the dataframe with the teams and their coefficients 
teams_List = []
scores_List = []
for i in range(400):
    teams_List.append(teams[i+1].text)
    scores_List.append(scores[i].text)
    
UEFA_club_coefficients = pd.DataFrame({"Team":teams_List,"Coefficient":scores_List})
UEFA_club_coefficients

,Team,Coefficient
0,Bayern München,136.000
1,Real Madrid,134.000
2,FC Barcelona,128.000
3,Atlético Madrid,127.000
4,Juventus,117.000
5,Manchester City,116.000
6,Paris Saint-Germain,113.000
7,Sevilla,102.000
8,Manchester United,100.000
9,Liverpool,99.000


### 2. League valuation using UEFA index

In [11]:
# No web scrapping needed here because we are only considering the top 5 leagues and it could be done manually
leagues = ['PremierLeague','SerieA','Bundesliga','Ligue1','LaLiga']
coefficients = [90.462,70.653,74.784,59.248,102.283]
UEFA_league_coefficients = pd.DataFrame({"League":leagues,"Coefficient":coefficients})
UEFA_league_coefficients.sort_values(by=["Coefficient"],inplace=True,ascending=False)
UEFA_league_coefficients

,League,Coefficient
4,LaLiga,102.283
0,PremierLeague,90.462
2,Bundesliga,74.784
1,SerieA,70.653
3,Ligue1,59.248


### 3. Nation valuation using FIFA index

In [12]:
# We scrapping on the official FIFA website to get the actual nations ranking 
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

page = "https://www.fifa.com/fifa-world-ranking/ranking-table/men/"
pageTree = requests.get(page, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

# Finding the values we are looking for
nations = pageSoup.find_all("div", {"class": "fi-t__n"})
points = pageSoup.find_all("td", {"class": "fi-table__td fi-table__points"})

# Creating the dataframe with the  and their coefficients 
nation_list=[]
points_list=[]
for i in range(len(nations)):
    nation = nations[i].text[1:]
    nation = nation[:nation.index('\n')] 
    nation_list.append(nation)
    points_list.append(points[i].text)
    
Nations_ranking = pd.DataFrame({"Nations" : nation_list, "Points":points_list})
Nations_ranking

,Nations,Points
0,Belgium,1780
1,France,1755
2,Brazil,1743
3,England,1670
4,Portugal,1662
5,Spain,1645
6,Argentina,1642
7,Uruguay,1639
8,Mexico,1632
9,Italy,1625


### 4. Replacing names

In [13]:
# Names need to be corrected because the nomenclature is different between web sources
# No other way to do it rather than manually

# For UEFA teams index 
DATA.replace({"BayernMunich":"BayernMünchen"}, inplace = True)
DATA.replace({"E.Frankfurt":"EintrachtFrankfurt"}, inplace = True)
DATA.replace({"Monaco":"ASMonaco"}, inplace = True)
DATA.replace({"Bor.Dortmund ":"BorussiaDortmund"}, inplace = True)
DATA.replace({"Bor.M'gladbach":"BorussiaMönchengladbach"}, inplace = True)
DATA.replace({"Athletic ":"AthleticBilbao"}, inplace = True)
DATA.replace({"1.FSVMainz05":"FSVMainz05"}, inplace = True)
DATA.replace({"FCSchalke04":"Schalke04"}, inplace = True)
DATA.replace({"Leicester":"LeicesterCity"}, inplace = True)
DATA.replace({"Burnley ":"BurnleyFC"}, inplace = True)
DATA.replace({"Marseille":"OlympiqueMarseille"}, inplace = True)
DATA.replace({"ParisSG":"ParisSaint-Germain"}, inplace = True)
DATA.replace({"Inter ":"Internazionale"}, inplace = True)
DATA.replace({"SevillaFC":"Sevilla"}, inplace = True)
DATA.replace({"R.Strasbourg":"RCStrasbourg"}, inplace = True)
DATA.replace({"G.Bordeaux":"GirondinsBordeaux"}, inplace = True)
DATA.replace({"ManUtd ":"ManchesterUnited"}, inplace = True)
DATA.replace({"ManCity":"ManchesterCity"}, inplace = True)
DATA.replace({"RapidVienna  ":"RapidWien"}, inplace = True)
DATA.replace({"SSCNapoli":"Napoli"}, inplace = True)
DATA.replace({"Bay.Leverkusen":"BayerLeverkusen"}, inplace = True)
DATA.replace({"LOSCLille ":"LilleOSC"}, inplace = True)
DATA.replace({"TSGHoffenheim":"1899Hoffenheim"}, inplace = True)
DATA.replace({"Saint0Étienne ":"ASSaint-Étienne"}, inplace = True)
DATA.replace({"Spurs":"TottenhamHotspur"}, inplace = True)
DATA.replace({"WestHam ":"WestHamUnited"}, inplace = True)
DATA.replace({"Southampton ":"SouthamptonFC"}, inplace = True)
DATA.replace({"Wolves ":"WolverhamptonWanderers"}, inplace = True)

# 2. For FIFA nations index
DATA.replace({"Coted'Ivoire":"Côte d'Ivoire"}, inplace = True)
DATA.replace({"UnitedStates":"USA"}, inplace = True)
DATA.replace({"CzechRepublic":"Czech Republic"}, inplace = True)
DATA.replace({"DRCongo":"Congo DR"}, inplace = True)
DATA.replace({"Bosnia0Herzegovina":"Bosnia and Herzegovina"}, inplace = True)
DATA.replace({"Korea,South":"Korea Republic"}, inplace = True)
DATA.replace({"SouthAfrica":"South Africa"}, inplace = True)
DATA.replace({"BurkinaFaso":"Burkina Faso"}, inplace = True)
DATA.replace({"China":"China PR"}, inplace = True)
DATA.replace({"Macedonia":"North Macedonia"}, inplace = True)
DATA.replace({"Iran":"IR Iran"}, inplace = True)
DATA.replace({"Ireland":"Republic of Ireland"}, inplace = True)
DATA.replace({"Curacao":"Curaçao"}, inplace = True)
DATA.replace({"EquatorialGuinea":"Equatorial Guinea"}, inplace = True)
DATA.replace({"Guinea0Bissau":"Guinea-Bissau"}, inplace = True)
DATA.replace({"TrinidadandTobago":"Trinidad and Tobago"}, inplace = True)
DATA.replace({"CentralAfricanRepublic":"Central African Republic"}, inplace = True)
DATA.replace({"SierraLeone":"Sierra Leone"}, inplace = True)

### 5. Replacing the strings in DATA by the corresponding value from the UEFA/FIFA indexes